In [1]:
import re
import string
import numpy as np
from nltk.tokenize import word_tokenize
import pandas as pd

In [2]:
corpus = np.array(["Sempre que ocorre um evento no cliente do Spotify, ele é enviado para um dos gateways do Spotify, que o registra via syslog. Lá, é atribuído um carimbo de data / hora que é usado em todo o sistema de entrega de eventos. todos os dados precisam ser entregues a um cluster Hadoop localizado centralmente. Todos os dados entregues por meio de nosso serviço de entrega de eventos são gravados em HDFS. O File Tailer controla os arquivos de log em busca de novos eventos e os encaminha para o Event Delivery Service. Assim que obtiver a confirmação de que o evento foi recebido, a responsabilidade termina. O Event Delivery Service aceita eventos do Tailer, transforma-os em seu formato estruturado final e os encaminha para os Kafka Brokers. Event Delivery Service é construído como um microsserviço RESTful usando a estrutura Apollo e implementado usando a plataforma de orquestração Helios, um padrão de design comum no Spotify. Incorporar um produtor Kafka simples no serviço de entrega de eventos também provou ser fácil. O projeto Mirror Maker introduziu o espelhamento entre data centers e o projeto Camus pode ser usado para exportar eventos estruturados Avro para depósitos de hora em hora.",
                   "Quase nada.",
                   
                   
                   
          ])

### 1) Aplicar lowercase e remover caracteres

In [3]:
def clear_text(text):
    pattern = "[{}]".format(string.punctuation)
    text = [word.lower() for word in text]
    text = [[re.sub(pattern, "", word) for word in words.split()] for words in text]
    text = [[word for word in words if len(word)>1] for words in text]    
    text = [' '.join(words) for words in text]
    return np.array(text)

In [4]:
corpus_clear = clear_text(corpus)
print(corpus_clear)

['sempre que ocorre um evento no cliente do spotify ele enviado para um dos gateways do spotify que registra via syslog lá atribuído um carimbo de data hora que usado em todo sistema de entrega de eventos todos os dados precisam ser entregues um cluster hadoop localizado centralmente todos os dados entregues por meio de nosso serviço de entrega de eventos são gravados em hdfs file tailer controla os arquivos de log em busca de novos eventos os encaminha para event delivery service assim que obtiver confirmação de que evento foi recebido responsabilidade termina event delivery service aceita eventos do tailer transformaos em seu formato estruturado final os encaminha para os kafka brokers event delivery service construído como um microsserviço restful usando estrutura apollo implementado usando plataforma de orquestração helios um padrão de design comum no spotify incorporar um produtor kafka simples no serviço de entrega de eventos também provou ser fácil projeto mirror maker introduzi

### 2) Remover stopwords

In [5]:
from nltk.corpus import stopwords
def clear_stopwords(text, language):
    filtered_text = []
    for t in text:
        words = [w for w in t.split() if not w in stopwords.words(language)]
        words = ' '.join(words)
        filtered_text.append(words)
    return np.array(filtered_text)   

In [6]:
corpus_filtered = clear_stopwords(corpus_clear, "french")
print(corpus_filtered)

['sempre ocorre um evento no cliente do spotify ele enviado para um dos gateways do spotify registra via syslog lá atribuído um carimbo data hora usado em todo sistema entrega eventos todos os dados precisam ser entregues um cluster hadoop localizado centralmente todos os dados entregues por meio nosso serviço entrega eventos são gravados em hdfs file tailer controla os arquivos log em busca novos eventos os encaminha para event delivery service assim obtiver confirmação evento foi recebido responsabilidade termina event delivery service aceita eventos do tailer transformaos em seu formato estruturado final os encaminha para os kafka brokers event delivery service construído como um microsserviço restful usando estrutura apollo implementado usando plataforma orquestração helios um padrão design comum no spotify incorporar um produtor kafka simples no serviço entrega eventos também provou ser fácil projeto mirror maker introduziu espelhamento entre data centers projeto camus pode ser us

### Usando a biblioteca sklearn

In [7]:
def create_dataframe(matrix, tokens):
    doc_names = [f'text_{i+1}' for i, _ in enumerate(matrix)]
    df = pd.DataFrame(data=matrix, index=doc_names, columns=tokens)
    return(df)

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorize = CountVectorizer()
vector_matrix = vectorize.fit_transform(corpus_filtered)

tokens = vectorize.get_feature_names()
create_dataframe(vector_matrix.toarray(),tokens)



,aceita,apollo,arquivos,assim,atribuído,avro,brokers,busca,camus,carimbo,...,tailer,também,termina,todo,todos,transformaos,um,usado,usando,via
text_1,1,1,1,1,1,1,1,1,1,1,...,2,1,1,1,2,1,7,2,2,1
text_2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
cosine_similarity_matrix = cosine_similarity(vector_matrix)
create_dataframe(cosine_similarity_matrix,['text_1','text_2'])

,text_1,text_2
text_1,1.0,0.0
text_2,0.0,1.0
